In [87]:
#Repeat scenario 1 results from paper Multiple Network Embedding for Anomaly Detection in Time Series of Graphs(link: file:///C:/Users/16827/OneDrive/Documents/Fall_2020_semester_jhu/NDD/Class_material/OMNIvsMASE.pdf)

#import all needed packages
import graspy

import heapq
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from graspy.simulations import rdpg
from numpy import linalg as LA
from graspy.inference import LatentPositionTest
from graspy.plot import heatmap, pairplot
from graspy.embed import OmnibusEmbed, select_dimension, AdjacencySpectralEmbed, MultipleASE
from scipy.linalg import orthogonal_procrustes

from graspy.utils import import_graph, is_symmetric
from graspy.inference import base

%matplotlib inline

In [88]:
# Copyright (c) Microsoft Corporation and contributors.
# Licensed under the MIT License.

import numpy as np
from abc import abstractmethod
from sklearn.base import BaseEstimator


class BaseInference(BaseEstimator):
    """
    Base class for inference tasks such as semiparametric latent position test
    and nonparametric latent distribution test.
    Parameters
    ----------
    n_components : None (default), or int
        Number of embedding dimensions. If None, the optimal embedding
        dimensions are chosen via the Zhu and Godsi algorithm.
    """

    def __init__(self, n_components=None):
        if (not isinstance(n_components, (int, np.integer))) and (
            n_components is not None
        ):
            raise TypeError("n_components must be int or np.integer")
        if n_components is not None and n_components <= 0:
            raise ValueError(
                "Cannot embed into {} dimensions, must be greater than 0".format(
                    n_components
                )
            )
        self.n_components = n_components

    @abstractmethod
    def _embed(self, A1, A2, n_componets):
        """
        Computes the latent positions of input graphs
        Parameters
        ----------
        A1 : np.ndarray, shape (n_vertices, n_vertices)
            Adjacency matrix of the first graph
        A2 : np.ndarray, shape (n_vertices, n_vertices)
            Adjacency matrix of the second graph
        Returns
        -------
        X1_hat : array-like, shape (n_vertices, n_components)
            Estimated latent positions of the vertices in the first graph
        X2_hat : array-like, shape(n_vertices, n_components)
            Estimated latent positions of the vertices in the second graph
        """

    @abstractmethod
    def fit(self, A1, A2):
        """
        Compute the test statistic and the null distribution.
        Parameters
        ----------
        A1, A2 : nx.Graph, nx.DiGraph, nx.MultiDiGraph, nx.MultiGraph, np.ndarray
            The two graphs to run a hypothesis test on.
            If np.ndarray, shape must be ``(n_vertices, n_vertices)`` for both
            graphs, where ``n_vertices`` is the same for both
        Returns
        ------
        self
        """
        pass

    def fit_predict(self, A1, A2):
        """
        Fits the model and returns the p-value
        Parameters
        ----------
        A1, A2 : nx.Graph, nx.DiGraph, nx.MultiDiGraph, nx.MultiGraph, np.ndarray
            The two graphs to run a hypothesis test on.
            If np.ndarray, shape must be ``(n_vertices, n_vertices)`` for both
            graphs, where ``n_vertices`` is the same for both
        Returns
        ------
        p_value_ : float
            The overall p value from the test
        """
        self.fit(A1, A2)
        return self.p_value_

In [89]:
#Create functions to compute the test statistics for GraphAD, VertexAD
def t_statistics_g(X, Y):
    '''This function accepts as input the latent position matrix of the two graphs for which the test statistic t is being computed
       According to the formula t = ||X - Y||(l2norm) where X indicates currrent graph and Y the previous graph in the time series
       and returns the computed t statistic'''
    y = LA.norm(X-Y, ord = 2)
    return y

def t_statistics_v(X, Y):
    '''This function accepts as input the latent position matrix of the two graphs for which the test statistic t is being computed.
       Then computes t-statistic for each vertex using latent position X(i) for the vertex i of graph X and Y(i) for the vertex i of graph Y.
       Graph X is latent position of current time series graph G(t) and Y is previous time G(t-1).
       The statistic is computed according to the formula:
       t(i) = ||X(i) - Y(i)||(l2-norm).
       Returns a tuple consisting of each individual vertex test statistic.
       Matrices X and Y must have the same dimensions'''
    y = ()
    for i in range(len(X)):
        y = y + (LA.norm(X[i]-Y[i], ord = 2),)
    return y

In [90]:
class LatentPositionTest_X(BaseInference):
    def __init__(
        self, embedding="ase", n_components=None, n_bootstraps=500, test_case="rotation"
    ):
        if type(embedding) is not str:
            raise TypeError("embedding must be str")
        if type(n_bootstraps) is not int:
            raise TypeError()
        if type(test_case) is not str:
            raise TypeError()
        if n_bootstraps < 1:
            raise ValueError(
                "{} is invalid number of bootstraps, must be greater than 1".format(
                    n_bootstraps
                )
            )
        if embedding not in ["ase", "omnibus"]:
            raise ValueError("{} is not a valid embedding method.".format(embedding))
        if test_case not in ["rotation", "scalar-rotation", "diagonal-rotation"]:
            raise ValueError(
                "test_case must be one of 'rotation', 'scalar-rotation',"
                + "'diagonal-rotation'"
            )

        super().__init__(n_components=n_components)

        self.embedding = embedding
        self.n_bootstraps = n_bootstraps
        self.test_case = test_case
        # paper uses these always, but could be kwargs eventually. need to test
        self.rescale = False
        self.loops = False

    def _bootstrap(self, X_hat):
        t_bootstrap = np.zeros(self.n_bootstraps)
        for i in range(self.n_bootstraps):
            A1_simulated = rdpg(X_hat, rescale=self.rescale, loops=self.loops)
            A2_simulated = rdpg(X_hat, rescale=self.rescale, loops=self.loops)
            X1_hat_simulated, X2_hat_simulated = self._embed(
                A1_simulated, A2_simulated, check_lcc=False
            )
            t_bootstrap[i] = self._difference_norm(X1_hat_simulated, X2_hat_simulated)
        return t_bootstrap
    
    def _embed(self, A1, A2, check_lcc=True):
        if self.embedding == "ase":
            X1_hat = AdjacencySpectralEmbed(
                n_components=self.n_components, check_lcc=check_lcc
            ).fit_transform(A1)
            X2_hat = AdjacencySpectralEmbed(
                n_components=self.n_components, check_lcc=check_lcc
            ).fit_transform(A2)
        elif self.embedding == "omnibus":
            X_hat_compound = OmnibusEmbed(
                n_components=self.n_components, check_lcc=check_lcc
            ).fit_transform((A1, A2))
            X1_hat = X_hat_compound[0]
            X2_hat = X_hat_compound[1]
        return (X1_hat, X2_hat)


    def _difference_norm(self, X1, X2):
        if self.embedding in ["ase"]:
            if self.test_case == "rotation":
                R = orthogonal_procrustes(X1, X2)[0]
                return np.linalg.norm(X1 @ R - X2)
            elif self.test_case == "scalar-rotation":
                R, s = orthogonal_procrustes(X1, X2)
                return np.linalg.norm(s / np.sum(X1 ** 2) * X1 @ R - X2)
            elif self.test_case == "diagonal-rotation":
                normX1 = np.sum(X1 ** 2, axis=1)
                normX2 = np.sum(X2 ** 2, axis=1)
                normX1[normX1 <= 1e-15] = 1
                normX2[normX2 <= 1e-15] = 1
                X1 = X1 / np.sqrt(normX1[:, None])
                X2 = X2 / np.sqrt(normX2[:, None])
                R = orthogonal_procrustes(X1, X2)[0]
                return np.linalg.norm(X1 @ R - X2)
        else:
            # in the omni case we don't need to align
            return np.linalg.norm(X1 - X2)
    
    def fit(self, X1, X2):
        """
        Fits the test to the two input graphs

        Parameters
        ----------
        A1, A2 : nx.Graph, nx.DiGraph, nx.MultiDiGraph, nx.MultiGraph, np.ndarray
            The two graphs to run a hypothesis test on.
            If np.ndarray, shape must be ``(n_vertices, n_vertices)`` for both graphs,
            where ``n_vertices`` is the same for both

        Returns
        -------
        self
        """
        X_hats = (X1, X2)
        sample_T_statistic = self._difference_norm(X_hats[0], X_hats[1])
        null_distribution_1 = self._bootstrap(X_hats[0])
        null_distribution_2 = self._bootstrap(X_hats[1])

        # uisng exact mc p-values (see, for example, Phipson and Smyth, 2010)
        p_value_1 = (
            len(null_distribution_1[null_distribution_1 >= sample_T_statistic]) + 1
        ) / (self.n_bootstraps + 1)
        p_value_2 = (
            len(null_distribution_2[null_distribution_2 >= sample_T_statistic]) + 1
        ) / (self.n_bootstraps + 1)

        p_value = max(p_value_1, p_value_2)

        self.null_distribution_1_ = null_distribution_1
        self.null_distribution_2_ = null_distribution_2
        self.sample_T_statistic_ = sample_T_statistic
        self.p_value_1_ = p_value_1
        self.p_value_2_ = p_value_2
        self.p_value_ = p_value

        return self


In [91]:
#define K-Neighrest neighboor function
def KNN(X, k):
    '''This function computes k nearest neighboors of rows of a matrix from the first row and outputs a tuple containing the rows of the knn. X must be a matrix'''
    distance = []
    for i in range(1,len(X)):
        dist = LA.norm(X[i]-X[i-1], ord = 2)
        distance.append(dist)
    smallest_k = heapq.nsmallest(k,((m, n) for n, m in enumerate(distance)))
    _, index_smallest =  zip(*smallest_k)
    index_smallest = np.array(index_smallest)
    return index_smallest
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [99]:
#Need to simulate MMSBM with B being a a dxd matrix with each entry being equal to q = 0.3 and diagonal being equal to p = 0.8. E is a dx1 vector with each entry being 0.3. Finally the Z matrix(n_vertices x d) is made up by Z(i) vector drawn from Dirichlet(theta* 1d) where theta is a parameter between 0 and 1 and 1d is a d-dimensional vector with d entries equal to 1. d = 4

from scipy.linalg import sqrtm

q = 0.3
p = 0.8
d = 4
theta = 1 #parameter to fine tune the MMSBM
k = 100 #knearest neighboor 

n = 400 #number of vertices
time_series = 12

np.random.seed(1)

B = np.full((d,d), q) - (np.array([q, q, q, q])) * np.identity(d)
diag_B = (np.array([p, p, p, p])) * np.identity(d)

B = B + diag_B

B_sqrt = sqrtm(B)
X_list = []

E = 0.11*np.ones((1,d))
G = []
Z_list = [] #list conytaining each Z
alpha = np.ones(d)*theta

for j in range(time_series):
    Z = []
    for i in range(n):
        z = np.random.dirichlet(alpha)
        Z.append(z)

    Z = np.array(Z)
    Z_list.append(Z)
    x = Z @ B_sqrt
    X_list.append(x)
    #g = np.random.binomial(1, Z @ B @ Z.T)
    g = rdpg(x)
    G.append(g)

X = np.array(X_list)
G = np.array(G)
#Now add perturbation at time 6 and 7

#To add perturbation find k=100 neighest neighboors of X(1) of X where X is the common latent position to all graphs besides perturbation and X(1) is the first row. For each row corresponding to one of the 100th neighrest neighboor the value of delta will be E, else it will be zero

index_to_pert = np.sort(KNN(X[0], k))

#Now that I have indexes to pertubate matrix, create delta by assigning a d-dimensional vector E to every row in the indexes found, else zero

delta = np.zeros(np.shape(X[0]))

for j in index_to_pert:
    delta[j] = E

#Now perturb time point 6

X[5] = X[5] + theta * delta #may need to change theta to a constant

#Now perturb time point 7

X[6] = X[6] - theta * delta  #here it may fail beacuse pertubations adds 0.3 and some entries are 0.856

print(X[6].shape)

#Finally add perturbed graph back into the time series of graphs

G[5] = rdpg(X[5])
G[6] = rdpg(X[6])

#print(((X[5] @ X[5].T)>1).sum())
#G[5] = np.random.binomial(1, X[5] @ X[5].T)
#G[6] = np.random.binomial(1, X[6] @ X[5].T)

#Now finally generated all the graphs

(400, 4)


In [101]:
#Embed according to algo 1
from scipy.linalg import sqrtm

def algo1(A, embedding, s=2):
    t=0
    Zhat_list = []
    while(t+s-1 <= len(A)-1):
        if(embedding == 'omnibus'):
            embedder = OmnibusEmbed()
            Zhat = embedder.fit_transform(A[t:s+t])
            Zhat_list.append(Zhat)
        elif(embedding == 'MASE'):
            embedder = MultipleASE()
            V = embedder.fit_transform(A[t:s+t])
            R = embedder.scores_ #For omni can jusr return R scores and use those to compute p-value
            X = []
            for i in range(len(R)):
                #R_sqrt = sqrtm(R[i])
                #x = V @ R_sqrt
                x = V @ R[i]
                X.append(x)
            X = np.array(X)
            Zhat_list.append(X)
                
        t = t+1
    return(Zhat_list)

#Embed with s = 12

MASE_embedding = algo1(G, embedding = 'MASE', s=2)
OMNI_embedding = algo1(G, embedding = 'omnibus', s=2)


MASE = MASE_embedding[5]
OMNI = OMNI_embedding[5]

print(MASE.shape)
print(MASE)

(2, 400, 2)
[[[ 9.73642491  0.28083816]
  [10.70244511 -0.26336759]
  [ 9.76597061  0.07133293]
  ...
  [10.17105778  0.05235695]
  [10.49966303  0.41793056]
  [ 8.85004468  0.15807448]]

 [[ 6.99719964 -0.23071342]
  [ 7.08483413  0.1321752 ]
  [ 6.79537553 -0.08955746]
  ...
  [ 7.05398491 -0.07848035]
  [ 7.66773511 -0.32640238]
  [ 6.25712676 -0.14416442]]]


In [ ]:
#now need to compute pvalues for time 6 and 7, in montecarlo fashion using number of simulations equal to 100
montecarlo = 50
sig_p = []

lpt_X = LatentPositionTest_X(n_bootstraps=10)
lpt_X = LatentPositionTest_X(n_bootstraps=100, n_components=d)

for i in range(montecarlo):
    lpt_X.fit(MASE[0],MASE[1])
    sig_p.append(lpt_X.p_value_)
    print('p = {}'.format(lpt_X.p_value_))

power_MASE = ((sig_p<=0.05).sum())/montecarlo
print(power_MASE)

sig_p = []

for i in range(montecarlo):
    lpt_X.fit(OMNI[0],OMNI[1])
    sig_p.append(lpt_X.p_value_)
    print('p = {}'.format(lpt_X.p_value_))

power_OMNI = ((sig_p<=0.05).sum())/montecarlo
print(power_OMNI)

In [98]:
lpt = LatentPositionTest(n_bootstraps=100, n_components=d, embedding='omnibus')
lpt.fit(G[5], G[6])
print('p = {}'.format(lpt.p_value_))

p = 0.009900990099009901
